# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 3:28PM,241354,10,PRF-32920,Bio-PRF,Released
1,Jul 7 2022 3:25PM,241353,10,0085842668,ISDIN Corporation,Released
2,Jul 7 2022 3:25PM,241353,10,0085842675,ISDIN Corporation,Released
3,Jul 7 2022 3:25PM,241353,10,0085842689,ISDIN Corporation,Released
4,Jul 7 2022 3:25PM,241353,10,0085842708,ISDIN Corporation,Released
5,Jul 7 2022 3:25PM,241353,10,0085842709,ISDIN Corporation,Released
6,Jul 7 2022 3:25PM,241353,10,0085842707,ISDIN Corporation,Released
7,Jul 7 2022 3:25PM,241353,10,0085842718,ISDIN Corporation,Released
8,Jul 7 2022 3:25PM,241353,10,0085842719,ISDIN Corporation,Released
9,Jul 7 2022 3:25PM,241353,10,0085842737,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,241348,Released,1
40,241349,Released,1
41,241351,Released,1
42,241353,Released,10
43,241354,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241348,NaN,1.0
241349,NaN,1.0
241351,NaN,1.0
241353,NaN,10.0
241354,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241244,0.0,1.0
241245,0.0,1.0
241246,0.0,1.0
241248,0.0,1.0
241249,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241244,0,1
241245,0,1
241246,0,1
241248,0,1
241249,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241244,0,1
1,241245,0,1
2,241246,0,1
3,241248,0,1
4,241249,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241244,,1
1,241245,,1
2,241246,,1
3,241248,,1
4,241249,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation
11,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc
12,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.
13,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc."
14,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc."
15,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation
64,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc."
65,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc."
66,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF,,1
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,,10
2,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,,1
3,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.,,1
4,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc.",,1
5,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",,1
6,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,,49
7,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",,1
8,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",,1
9,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF,1,
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,10,
2,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,1,
3,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.,1,
4,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc.",1,
5,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",1,
6,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,49,
7,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",1,
8,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",1,
9,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF,1,
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,10,
2,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,1,
3,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.,1,
4,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF,1.0,NaN
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,10.0,NaN
2,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,1.0,NaN
3,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.,1.0,NaN
4,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 3:28PM,241354,10,Bio-PRF,1.0,0.0
1,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,10.0,0.0
2,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,1.0,0.0
3,Jul 7 2022 3:01PM,241349,20,Alumier Labs Inc.,1.0,0.0
4,Jul 7 2022 2:54PM,241348,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4343214,306.0,21.0
102,965249,4.0,0.0
16,241256,1.0,0.0
18,241351,1.0,0.0
20,965285,80.0,5.0
21,2654212,10.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4343214,306.0,21.0
1,102,965249,4.0,0.0
2,16,241256,1.0,0.0
3,18,241351,1.0,0.0
4,20,965285,80.0,5.0
5,21,2654212,10.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,306.0,21.0
1,102,4.0,0.0
2,16,1.0,0.0
3,18,1.0,0.0
4,20,80.0,5.0
5,21,10.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,306.0
1,102,Released,4.0
2,16,Released,1.0
3,18,Released,1.0
4,20,Released,80.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,18,20,21
Status,,,,,,
Executing,21.0,0.0,0.0,0.0,5.0,1.0
Released,306.0,4.0,1.0,1.0,80.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,18,20,21
0,Executing,21.0,0.0,0.0,0.0,5.0,1.0
1,Released,306.0,4.0,1.0,1.0,80.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,18,20,21
0,Executing,21.0,0.0,0.0,0.0,5.0,1.0
1,Released,306.0,4.0,1.0,1.0,80.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()